# 类型
* 静态类型：在程序运行之前，我们就可计算每一个表达式的类型。
  * 通过让编写的代码无需在编译时知道值的确切类型，面向对象允许静态类型语言具有一定的灵活性。可以编写在不同类型上都能运行的代码的能力被称为多态。 
* 动态类型：只有通过运行那个程序，得到表达式具体的值，才能确定其具体的类型。

Julia 类型系统是动态的，但通过允许指出某些变量具有特定类型，获得了静态类型系统的一些优点。这对于生成高效的代码非常有帮助，但更重要的是，**它允许针对函数参数类型的方法分派与语言深度集成。**方法分派将在方法中详细探讨，但它根植于此处提供的类型系统。当需要额外的表达力时，很容易逐渐将显式的类型注释引入先前的「无类型」代码中。添加类型注释主要有三个目的：**利用 Julia 强大的多重分派机制、提高代码可读性以及捕获程序错误。**

**Julia 用类型系统的术语描述是动态（dynamic）、主格（nominative）和参数（parametric）的。**范型可以被参数化，并且类型之间的层次关系可以被**显式地声明**，而不是隐含地通过兼容的结构。 Julia 类型系统的一个特别显著的特征是具体类型相互之间不能是子类型：所有具体类型都是最终的类型，并且只有抽象类型可以作为其超类型。能够继承行为比继承结构更重要，同时继承两者在传统的面向对象语言中导致了重大困难。<br/>
* 对象值和非对象值之间没有分别。Julia中的所有值都是具有类型的真实对象。
* 编译期类型是没有任何意义的概念：变量所具有的唯一类型是程序运行时的实际类型。这在面向对象中被称为“运行时类型”。其中静态编译和多态的组合使得这种区别变得显著。
* 值有类型，变量没有类型。
* 抽象类型和具体类型都可以通过其他类型参数化。
  * 它们的参数化还可以通过符号、任意使得isbits()返回true的类型的值和元组。类型参数在不需要被引用或限制时可以省略。
  
## 类型声明
**::运算符可以用来在程序中给表达式和变量附加类型注释**，当被附加到一个计算值的表达式时，::代表“一个...的实例”，当右侧类型是具体类型时，左侧的值必须能够以该类型作为其实现，否则报错。当被附加到赋值左侧的变量或作为 local 声明的一部分时，:: 操作符的意义有所不同：它**声明变量始终具有指定的类型**.这个特性用于避免给一个变量赋值时意外更改了类型。**声明也可以附加到函数定义.**
```Julia
(1+2)::AbstractFloat#报错

(1+2)::Int#return 3

function foo()
    x::Int8=100
    x
end
foo()#100

local x::Int8
x::Int8=10

function sinc(x)::Float64
    if x==0
        return 1
    end
    return sin(pi*x)/(pi*x)
end

```
## 抽象类型
抽象类型不能实例化，只能作为类型图中的节点使用，从而描述由相关具体类型组成的集合。使用**abstract type**关键字来声明抽象类型。跟 **<: **和一个已存在的类型，表示新声明的抽象类型是此「父」类型的子类型。
```Julia
abstract type <<name>> end
abstract type <<name>> <: <<supertype>> end
```
如果没有给出超类型，则默认超类型为** Any——一个预定义的抽象类型，所有对象都是它的实例并且所有类型都是它的子类型**。在类型理论中，Any 通常称为"「top」，因为它位于类型图的顶点。Julia还有一个预定义的抽象「bottom」类型，在类型图的最低点，写成 Union{}。这与 Any 完全相反：**任何对象都不是 Union{} 的实例，所有的类型都是 Union{} 的超类型**。<br/>

构成 Julia 数值类型层次结构的抽象类型:
```Julia
abstract type Number end
abstract type Real <: Number end
abstract type AbstractFloat <: Real end
abstract type Integer <: Real end
abstract type Singed <: Integer end
abstract type Unsigned <: Integer end
```
Number 类型为 Any 类型的直接子类型，并且 Real 为它的子类型。反过来，Real 有两个子类型（它还有更多的子类型，但这里只展示了两个，稍后将会看到其它的子类型）： Integer 和 AbstractFloat，将世界分为整数的表示和实数的表示。实数的表示当然包括浮点类型，但也包括其他类型，例如有理数。因此，AbstractFloat 是一个 Real 的子类型，仅包括实数的浮点表示。整数被进一步细分为 Signed 和 Unsigned 两类。<br/>
```Julia
Integer <: Number#true
Integer <: AbstractFloat#false
```

```Julia
function myplus(x,y)
    x+y
end
```
需要注意的是上述的参数声明等价于 x::Any 和 y::Any。当函数被调用时，例如 myplus(2,5)，分派器选择与给定参数相匹配的名称为 myplus 的最具体方法。假设没有找到比上述方法更具体的方法，Julia 接下来会在内部定义并编译一个名为 myplus 的方法，专门用于基于上面给出的泛型函数的两个 Int 参数.
## 原始类型
原始类型是具体类型，其数据是由简单的位组成。原始类型的经典示例是整数和浮点数。与大多数语言不同，Julia 允许你声明自己的原始类型，而不是只提供一组固定的内置原始类型。实际上，标准原始类型都是在语言本身中定义的：
```Julia
primitive type Float32 <: AbstractFloat 32 end
primitive type Bool <: Integer 8 end
primitive type Char <: AbstractChar 32 end
primitive type Int8 <: Signed 8 end
primitive type UInt8 <: Unsigned 8 end
```
其一般语法为：
``` Julia
primitive type <<name>> <<bits>> end
primitive type <<name>> <: <<supertype>> <<bits>> end
```
bits 的数值表示该类型需要多少存储空间，name 为新类型指定名称。可以选择将一个原始类型声明为某个超类型的子类型。如果省略超类型，则默认 Any 为其直接超类型。上述声明中意味着 Bool 类型需要 8 位来储存，并且直接超类型为 Integer。目前支持的大小只能是 8 位的倍数。<br/>
Bool，Int8 和 UInt8 类型都具有相同的表现形式：它们都是 8 位内存块。然而，由于 Julia 的类型系统是主格的，它们尽管具有相同的结构，但不是通用的。它们之间的一个根本区别是它们具有不同的超类型：Bool 的直接超类型是 Integer、Int8 的是 Signed 而 UInt8 的是 Unsigned。Bool，Int8 和 UInt8 的所有其它差异是行为上的——定义函数的方式在这些类型的对象作为参数给定时起作用。这也是为什么主格的类型系统是必须的：如果结构确定类型，类型决定行为，就不可能使 Bool 的行为与 Int8 或 UInt8 有任何不同。
## 复合类型
复合类型在各种语言中被称为 record、struct 和 object。复合类型是命名字段的集合，其实例可以视为单个值。复合类型在许多语言中是唯一一种用户可定义的类型，也是 Julia 中最常用的用户定义类型。<br/>
在主流的面向对象语言中，比如 C++、Java、Python 和 Ruby，复合类型也具有与它们相关的命名函数，并且该组合称为「对象」。**在纯粹的面向对象语言中，例如 Ruby 或 Smalltalk，所有值都是对象，无论它们是否为复合类型。在不太纯粹的面向对象语言中，包括 C++ 和 Java，一些值，比如整数和浮点值，不是对象，而用户定义的复合类型是具有相关方法的真实对象。在 Julia 中，所有值都是对象，但函数不与它们操作的对象捆绑在一起。**这是必要的，因为 Julia 通过多重分派选择函数使用的方法，这意味着在选择方法时考虑所有函数参数的类型，而不仅仅是第一个（有关方法和分派的更多信息，请参阅方法）。因此，函数仅仅「属于」它们的第一个参数是不合适的。将方法组织到函数对象中而不是在每个对象「内部」命名方法最终成为语言设计中一个非常有益的方面。<br/>

struct 关键字与复合类型一起引入，后跟一个字段名称的块，可选择使用 :: 运算符注释类型：
```Julia
struct Foo
    bar#没有类型注释的字段默认为 Any 类型，所以可以包含任何类型的值。
    baz::Int
    qux::Float64
end
foo =Foo("A",23,1.5)
typeof(foo)
```
当像函数一样使用类型时，它被称为构造函数。有两个构造函数会被自动生成（这些构造函数称为默认构造函数）。 一个接受任何参数并通过调用 convert 函数将它们转换为字段的类型，另一个接受与字段类型完全匹配的参数。两者都生成的原因是，这使得更容易添加新定义而不会在无意中替换默认构造函数。<br/>

* 使用 struct 声明的对象都是不可变的，它们在构造后无法修改:
  * 它可以更高效。某些 struct 可以被高效地打包到数组中，并且在某些情况下，编译器可以完全避免分配不可变对象。
  * 不可能违反由类型的构造函数提供的不变性。
  * 使用不可变对象的代码更容易推理。
    
* 如果需要，可以使用关键字 mutable struct 声明可变复合对象.

### 可变复合类型

为了支持修改，这种对象通常分配在堆上，并且具有稳定的内存地址。**可变对象就像一个小容器，随着时间的推移，可能保持不同的值，因此只能通过其地址可靠地识别。相反地，不可变类型的实例与特定字段值相关——仅字段值就告诉你该对象的所有内容。**在决定是否使类型为可变类型时，请询问具有相同字段值的两个实例是否被视为相同，或者它们是否可能需要随时间独立更改。如果它们被认为是相同的，该类型就应该是不可变的。<br/>
* 不允许修改不可变类型的值
  * 对于位类型，这意味着值的位模式一旦设置将不再改变，并且该值是位类型的标识。
  * 对于复合类型，这意味着其字段值的标识将不再改变。当字段是位类型时，这意味着它们的位将不再改变，对于其值是可变类型（如数组）的字段，这意味着字段将始终引用相同的可变值，尽管该可变值的内容本身可能被修改。
  
* 具有不可变类型的对象可以被编译器自由复制，因为其不可变性使得不可能以编程方式区分原始对象和副本。
  * 特别地，这意味着足够小的不可变值（如整数和浮点数）通常在寄存器（或栈分配）中传递给函数。
  * 另一方面，~~可变值是堆分配的，并作为指向堆分配值的指针传递给函数，除非编译器确定没有办法知道这不是正在发生的事情。~~
 
### 已声明的类型
抽象、原始、复合，在内部表现为相同概念 DataType 的实例。
```Julia
typeof(Real)#Datatype
typeof(Int)#DataType
```
DataType 可以是抽象的或具体的。它如果是具体的，就具有指定的大小、存储布局和字段名称（可选）。因此，原始类型是具有非零大小的 DataType，但没有字段名称。复合类型是具有字段名称或者为空（大小为零）的 DataType。
### 类型共用体
类型共用体是一种特殊的抽象类型，它包含作为对象的任何参数类型的所有实例，使用特殊Union关键字构造：

```Julia
IntOrString=Union{Int,AbstractString}
1::IntOrString
"Hello!" :: IntOrString
1.0 :: IntOrString
```

Julia 编译器能在 Union 类型只具有少量类型的情况下生成高效的代码，方法是**为每个可能类型的不同分支都生成专用代码**。<br/>

Union 类型的一种特别有用的情况是 Union{T, Nothing}，其中 T 可以是任何类型，Nothing 是单态类型，其唯一实例是对象 nothing。此模式是其它语言中 Nullable、Option 或 Maybe 类型在 Julia 的等价。通过将函数参数或字段声明为 Union{T, Nothing}，可以将其设置为类型为 T 的值，或者 nothing 来表示没有值。

## 参数类型
Julia 类型系统的一个重要和强大的特征是它是参数的：类型可以接受参数，因此类型声明实际上引入了一整套新类型——每一个参数值的可能组合引入一个新类型。许多语言支持某种版本的“泛型编程”：可以指定操作泛型的数据结构和算法，而无需指定所涉及的确切类型。其中，ML、Haskell、Scala等支持真正的参数多态，而C++和JAVA等支持基于模板的泛型编程。所有已声明的类型（DataType 类型）都可被参数化，在每种情况下都使用一样的语法。
### 参数复合类型

```Julia
struct Point{T}
    x::T
    y::T
end
```
此声明定义了一个新的参数类型，Point{T},它可以是任何类型（或者任何位类型值，虽然它实际上在这里显然用作类型）,Point{Float64} 是一个具体类型，该类型等价于通过用 Float64 替换 Point 的定义中的 T 所定义的类型.<br/>
***
Point 本身也是一个有效的类型对象，包括所有实例 Point{Float64}、Point{AbstractString} 等作为子类型<br/>
***
Point 不同 T 值所声明的具体类型之间，不能互相作为子类型
***
-----------------------------------



```Julia
Point{Float64}
Point{AbstractString}
Point{Float64} <:Point#true
Point{AbstractString} <: Point#true
```
* **即使 Float64 <: Real 也没有 Point{Float64} <: Point{Real}。**由于 Real 实例的对象可以具有任意的大小和结构，Point{Real} 的实例实际上必须表示为一对指向单独分配的 Real 对象的指针。<br/>


```Julia
function norm(p::Point{Real})#不适用于类型为 Point{Float64} 的参数
    sqrt(p.x^2+p.y^2)
end

function norm(p::Point{<:Real})#定义一个接受类型的所有参数的方法，Point{T}其中#T是一个子类型Real
    sqrt(p.x^2+p.y^2)
end

###等价于
function norm(p::Point{T} where T<:Real)
    sqrt(p.x^2+p.y^2)
end

###或
function norm(p::Point{T}) where T<:Real
    sqrt(p.x^2+p.y^2)
end
```

可以为复合类型定义自定义的构造函数，这将在构造函数中详细讨论，但在没有任何特别的构造函数声明的情况下，有两种默认方式可以创建新的复合对象，一种是显式地给出类型参数，另一种是通过传给对象构造函数的参数隐含地给出。

```Julia
Point{Float64}(1.0,2.0)
typeof(ans)#Point{Float64}
```
* 对于默认的构造函数，必须为每个字段提供一个参数,既不能多，也不能少。

因为构造函数调用参数的类型已经隐式地提供了类型信息。因此，你也可以将 Point 本身用作构造函数，前提是参数类型 T 的隐含值是确定的:

```Julia
Point(1.0,2.0)#Point{Float64}(1.0,2.0)
typeof(ans)#Point{Float64}
Point(1,2)
typeof(ans)#Point{Int64}
```
* 当且仅当Point的两个参数类型相同时，T的类型才是隐含的，否则将报错，解决的办法是定义混合类型的构造函数。

### 参数抽象类型

```Julia
abstract type Pointy{T} end
#与参数复合类型一样，每个此类型的实例都是 Pointy 的子类型
Pointy{Int64}<:Pointy
Pointy{1}<:Pointy
#不同的参数抽象类型互不为子集。
#但支持协变和逆变类型
Pointy{Float64}<:Pointy{<:Real}#true
Pointy{Real}<:Pointy{>:Int}#true

#正如之前的普通抽象类型用于在具体类型上创建实用的类型层次结构一样
#参数抽象类型在参数复合类型上具有相同的用途
struct Point{T}<:Pointy{T}
    x::T
    y::T
end
#鉴于此类声明，对每个 T，都有 Point{T} 是 Pointy{T} 的子类型
Point{Float64}<:Pointy{Float64}#true
Point{Real}<:Pointy{Real}#true
Point{AbstractString}<:Pointy{AbstractString}#true
#但依然有
Point{Float64}<:Pointy{Real}#false
Point{Float64}<:Pointy{<:Real}#ture
##允许对被所有 Pointy 对象共享的公共接口进行编程，
##接口都由 Point 和 DiagPoint 实现。

#有时，类型参数取遍所有可能类型也许是无意义的,因此需施加约束
abstract type Pointy{T<:Real}end#可以使用任何 Real 的子类型替换 T

#参数化复合类型的类型参数可用相同的方式限制：
struct Point{T<:Real}<:Pointy{T}
    x::T
    y::T
end
#Julia 的不可变类型 Rational 的实际定义（为了简单起见省略了的构造函数）
struct Rational{T<:Integer}<:Real
    num::T
    den::T
end
```
#### 元组类型

元组类型是函数参数的抽象——不是函数本身的。函数参数的突出特征是它们的顺序和类型。因此，元组类型类似于参数化的不可变类型，其中每个参数都是一个字段的类型。
```Julia
struct Tuple2{A,B}
    a::A
    b::B
end

typeof((1,"foo",2.5))#Tuple{Int64,String,Float64}

Tuple{Int,AbstractString}<:Tuple{Real,Any}#true
Tuple{Int,AbstractString}<:Tuple{Real,Real}#false
Tuple{Int,AbstractString}<:Tuple{Real,}#false
```
* 元组类型可以具有任意数量的参数。
* 元组类型的参数是协变的：Tuple{Int} 是 Tuple{Any} 的子类型。因此，Tuple{Any} 被认为是一种抽象类型，且元组类型只有在它们的参数都是具体类型时才是具体类型。
* 元组没有字段名称; 字段只能通过索引访问。
#### 变参元组类型
元组类型的最后一个参数可以是特殊类型 Vararg，它表示任意数量的尾随参数

```Julia
mytupletype=Tuple{AbstractString,Vararg{Int}}
#Vararg{T} 对应于零个或更多的类型为 T 的元素
isa(("1",),mytupletype)#true
isa(("1",1,1,1),mytupletype)#true
#类型 Vararg{T,N} 对应于正好 N 个类型为 T 的元素。
```

#### 具名元组类型
具名元组是 NamedTuple 类型的实例，该类型有两个参数：一个给出字段名称的符号元组，和一个给出字段类型的元组类型。
```Julia
#NamedTuple 类型可以用作构造函数，接受一个单独的元组作为参数。
#构造出来的 NamedTuple 类型可以是具体类型，如果参数都被指定，
#也可以是只由字段名称所指定的类型：
typeof((a=1,b="hello"))#NamedTuple{(:a,:b),Tuple{Int64,String}}
NamedTuple{(:a,:b),Tuple{Float32,String}}((1,""))#(a=1.0f0,b="")
NamedTuple{(:a,:b)}((1,""))#(a=1,b="")
#如果指定了字段类型，参数会被转换。否则，就直接使用参数的类型。
```
#### 单态类型
```Julia
#一种特殊的抽象类型：单态类型。对于每个类型 T，
#「单态类型」Type{T} 是个抽象类型且唯一的实例就是对象 T.
isa(Float64,Type{Float64})#true
isa(Real,Type{Float64})#false
isa(Float64,Type{Real})#false
#isa(A,Type{B}) 为真当且仅当 A 与 B 是同一对象且该对象是一个类型。
#不带参数时，Type 是个抽象类型，所有类型对象都是它的实例，当然也包括单态类型
isa(Float64,Type)#true
isa(Type{Float64},Type)#true
isa(1,Type)#false,类型的类型
```
### 参数原始类型
```Julia
#原始类型也可以参数化声明，例如，指针都能表示为原始类型
primitive type Ptr{T} 32 end
primitive type Ptr{T} 64 end
#类型参数 T 并未在类型本身的定义里使用——它实际上只是一个抽象的标记，
#定义了一整族具有相同结构的类型，类型间仅由它们的类型参数来区分。
#因此，Ptr{Float64} 和 Ptr{Int64} 是不同的类型，就算它们具有相同的表示。
#当然，所有特定的指针类型都是总类型 Ptr 的子类型：
Ptr{Float64}<:Ptr
Ptr{Int64}<:Ptr
```
## UnionAll
答案是 Ptr（或其它参数类型像 Array）是一种不同种类的类型，称为 UnionAll 类型。这种类型表示某些参数的所有值的类型的迭代并集。

UnionAll 类型通常使用关键字 where 编写。例如，Ptr 可以更精确地写为 Ptr{T} where T，也就是对于 T 的某些值，所有类型为 Ptr{T} 的值。在这种情况下，参数 T 也常被称为「类型变量」，因为它就像一个取值范围为类型的变量。每个 where 只引入一个类型变量，因此在具有多个参数的类型中这些表达式会被嵌套，例如 Array{T,N} where N where T。

类型应用语法 A{B,C} 要求 A 是个 UnionAll 类型，并先把 B 替换为 A 中最外层的类型变量。结果应该是另一个 UnionAll 类型，然后把 C 替换为该类型的类型变量。所以 A{B,C} 等价于 A{B}{C}。这解释了为什么可以部分实例化一个类型，比如 Array{Float64}：第一个参数已经被固定，但第二个参数仍取遍所有可能值。通过使用 where 语法，任何参数子集都能被固定。例如，所有一维数组的类型可以写为 Array{T,1} where T。

类型变量可以用子类型关系来加以限制。Array{T} where T<:Integer 指的是元素类型是某种 Integer 的所有数组。语法 Array{<:Integer} 是 Array{T} where T<:Integer 的便捷的缩写。类型变量可同时具有上下界。Array{T} where Int<:T<:Number 指的是元素类型为能够包含 Int 的 Number 的所有数组（因为 T 至少和 Int 一样大）。语法 where T>:Int 也能用来只指定类型变量的下界，且 Array{>:Int} 等价于 Array{T} where T>:Int。

由于 where 表达式可以嵌套，类型变量界可以引用更外层的类型变量。比如 Tuple{T,Array{S}} where S<:AbstractArray{T} where T<:Real 指的是二元元组，其第一个元素是某个 Real，而第二个元素是任意种类的数组 Array，且该数组的元素类型包含于第一个元组元素的类型。

where 关键字本身可以嵌套在更复杂的声明里。例如，考虑由以下声明创建的两个类型：

```Julia
const T1=Array{Array{T,1} where T,1}
const T2=Array{Array{T,1},1} where T
```
类型 T1 定义了由一维数组组成的一维数组；每个内部数组由相同类型的对象组成，但此类型对于不同内部数组可以不同。另一方面，类型 T2 定义了由一维数组组成的一维数组，其中的每个内部数组必须具有相同的类型。请注意，T2 是个抽象类型，比如 Array{Array{Int,1},1} <: T2，而 T1 是个具体类型。因此，T1 可由零参数构造函数 a=T1() 构造，但 T2 不行。

命名此类型有一种方便的语法，类似于函数定义语法的简短形式：
```Julia
Vector{T}=Array{T,1}
```
这等价于 const Vector = Array{T,1} where T。编写 Vector{Float64} 等价于编写 Array{Float64,1}，总类型 Vector 具有所有 Array 对象的实例，其中 Array 对象的第二个参数——数组维数——是 1，而不考虑元素类型是什么。在参数类型必须总被完整指定的语言中，这不是特别有用，但在 Julia 中，这允许只编写 Vector 来表示包含任何元素类型的所有一维密集数组的抽象类型。

### 类型别名

有时为一个已经可表达的类型引入新名称是很方便的。这可通过一个简单的赋值语句完成。
```Julia
UInt#UInt32,for 32-bit system
UInt#UInt64,for 64-bit system
#在 base/boot.jl 中，通过以下代码实现：
if Int === Int64
    const UInt=Uint64
else
    const UInt=UInt32
    end
#Float 不作为特定大小的 AbstractFloat 类型的别名而存在。
#与整数寄存器不同，浮点数寄存器大小由 IEEE-754 标准指定。
#而 Int 的大小反映了该机器上本地指针的大小。
```




### 类型操作
因为 Julia 中的**类型本身就是对象**，所以一般的函数可以对它们进行操作。已经引入了一些对于使用或探索类型特别有用的函数，例如 <: 运算符，它表示其左操作数是否为其右操作数的子类型。
```Julia
typeof(Rational{Int})#DataType
typeof(Union{Real,Float64,Rational})#DataType
typeof(Union{Real,String})#Union
typeof(Union)#DataType
typeof(DataType)#DataType
#另一个适用于某些类型的操作是 supertype，它显示了类型的超类型。
#只有已声明的类型（DataType）才有明确的超类型
supertype(Float32)#AbstractFloat
supertype(Number)#Any
supertype(AbstractString)#Any
supertype(Any)#Any
supertype(Union{Float64,Int64})#报错

```
## 自定义pretty-printing
要自定义显示类型实例的方式。这可通过重载 show 函数来完成。举个例子，假设我们定义一个类型来表示极坐标形式的复数：

```Julia
struct Polar{T<:Real}<:Number
    r::T
    theta::T
end
Polar(r::Real,theta::Real)=Polar(promote(r,theta)...)
#添加自定义的构造函数接受不同 Real 类型的参数并将它们类型提升为共同类型
#如果我们希望它显示为 3.0 * exp(4.0im)，
#我们将定义以下方法来将对象打印到给定的输出对象 io
Base.show(io::IO,z::Polar)=print(io,z.r,"*exp(",z.theta,"im)")

#可以定义一个不同的多行格式来显示单个对象，这通过重载三参数形式的 show 函数，
#该函数接收 text/plain MIME 类型（请参阅 多媒体 I/O）作为它的第二个参数

Base.show(io::IO,::MIME"text/plain",z::Polar{T}) where{T}= 
print(io,"Polar{$T} complex number :\n ",z)
```
请注意 print(..., z) 在这里调用的是双参数的 show(io, z) 方法,从技术上讲，REPL 调用 display(z) 来显示单行的执行结果，其默认为 show(stdout, MIME("text/plain"), z)，而后者又默认为 show(stdout, z)，但是你不应该定义新的 display 方法，除非你正在定义新的多媒体显示管理器


此外，你还可以为其它 MIME 类型定义 show 方法，以便在支持的环境（比如 IJulia）中实现更丰富的对象显示（HTML、图像等）。例如，我们可以定义 Polar 对象的 HTML 显示格式，使其带有上标和斜体：


之后会在支持 HTML 显示的环境中自动使用 HTML 显示 Polar 对象，但如果你想，也可以手动调用 show 来获取 HTML 输出：


### 值类型
在 Julia 中，你无法根据诸如 true 或 false 之类的值进行分派。然而，你可以根据参数类型进行分派，Julia 允许你包含「plain bits」值（类型、符号、整数、浮点数和元组等）作为类型参数。Array{T,N} 里的维度参数就是一个常见的例子，在那里 T 是类型（比如 Float64），而 N 只是个 Int。

你可以创建把值作为参数的自定义类型，并使用它们控制自定义类型的分派。为了说明这个想法，让我们引入参数类型 Val{x} 和构造函数 Val(x) = Val{x}()，它可以作为一种习惯的方式来利用这种技术需要更精细的层次结构。这可以作为利用这种技术的惯用方式，而且不需要更精细的层次结构。

Val 的定义为：
``` Julia
struct Val{x}
end
Val(x)=Val{x}()#Val
firstlast(::Val{true}) = "First"
firstlast(::Val{false}) = "Last"
firstlast(Val(true))
firstlast(Val(false))
```
调用处应当始终传递 Val 实例而不是类型，也就是使用 foo(Val(:bar)) 而不是 foo(Val{:bar})。


In [1]:
struct Foo
    bar#没有类型注释的字段默认为 Any 类型，所以可以包含任何类型的值。
    baz::Int
    qux::Float64
end
foo =Foo("A",23,1.5)
typeof(foo)

#############
fieldnames(Foo)#(:bar),(:baz),(:qux)
foo.bar#可以使用传统的 foo.bar 表示法访问复合对象的字段值


#没有字段的不可变复合类型是单态类型；这种类型只能有一个实例：
struct NoFields
end
Nofield() === NoFields()
#  ‘’=== 函数用来确认构造出来的「两个」NoFields 实例实际上是同一个。

#如果使用mutable struct而不是struct声明复合类型，则它的实例可以被修改。

mutable struct Bar
    baz
    qux::Float64
end

bar=Bar("Hello",1.5)
bar.qux=2.0
bar.baz=1//2





Foo

In [6]:
struct Point{T}
    x::T
    y::T
end
function norm(p::Point{T} where T<:Real)
    sqrt(p.x^2+p.y^2)
end

norm (generic function with 1 method)

In [11]:
struct Polar{T<:Real}<:Number
    r::T
    theta::T
end
Polar(r::Real,theta::Real)=Polar(promote(r,theta)...)
#添加自定义的构造函数接受不同 Real 类型的参数并将它们类型提升为共同类型
#如果我们希望它显示为 3.0 * exp(4.0im)，
#我们将定义以下方法来将对象打印到给定的输出对象 io
Base.show(io::IO,z::Polar)=print(io,z.r," *exp(",z.theta,"im)")

#可以定义一个不同的多行格式来显示单个对象，这通过重载三参数形式的 show 函数，
#该函数接收 text/plain MIME 类型（请参阅 多媒体 I/O）作为它的第二个参数
#############################
Base.show(io::IO, ::MIME"text/plain", z::Polar{T}) where{T}=
print(io,"Polar{$T} complex number :\n ",z)
Polar(3,4.0)
[Polar(3,4.0),Polar(4.0,5.3)]
###################
Base.show(io::IO, ::MIME"text/html", z::Polar{T}) where {T} =
           println(io, "<code>Polar{$T}</code> complex number: ",
                   z.r, " <i>e</i><sup>", z.Θ, " <i>i</i></sup>")
############
show(stdout, "text/html", Polar(3.0,4.0))

In [ ]:
struct Val{x}
end
Val(x)=Val{x}()#Val
firstlast(::Val{true}) = "First"
firstlast(::Val{false}) = "Last"
firstlast(Val(true))
firstlast(Val(false))